In [9]:
import os
from datetime import datetime
import h5py
import numpy as np
import pandas as pd
from sklearn import preprocessing
from data_helpers import format_timeseries
from tempConv import tempConvDecoder

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
conf = {
    'bs' : 256,
    'eps' : 4,
    'lr' : 0.0005,
    'kernel' : 2,
    'nb_filter' : 5,
    'window' : 50,
    'offset' : -20,
    'regressor' : True,
    'pyramidal' : True,
    'resample_data' : False,
    'sample_size' : 40000,
    'run_id' : datetime.now(),
    'verbose' : False,
    'key' : ['absolute pitch'],
    'dataset' : None
}

In [13]:
## produce dataset_paths, format: [[path/to/X1, path/to/y1], [path/to/X2, path/to/y2]..]
## example of collecting some datasets into a list of tuples
rat_path = 'datasets/GRat31/'
data_keys = [folder for folder in os.listdir('datasets/GRat31/')]
folders = [rat_path+folder for folder in os.listdir('datasets/GRat31/')]

X_fname = 'mua_firing_rates_100hz.hdf5'
y_fname = 'all_head_data_100hz.hdf5'
dataset_paths = []
for folder in folders:
    data_file_list = os.listdir(folder)
    dataset_paths.append([
        folder+'/'+data_file_list[data_file_list.index(X_fname)],
        folder+'/'+data_file_list[data_file_list.index(y_fname)]
    ])

# lets train on the first dataset and test on the second two
test_paths = [dataset_paths.pop(0)]
test_paths, dataset_paths

X_train, y_train = format_timeseries(
    dataset_paths, 
    conf['window'],
    conf['offset'],
    (0,4,1), 
    (0,1),
    sample_frac=.7
)

X_test, y_test = format_timeseries(
    test_paths, 
    conf['window'],
    conf['offset'],
    (0,4,1), 
    (0,1)
)

dataset = {
    'train' : (X_train, y_train),
    'test' : (X_test, y_test)
}

conf['dataset'] = dataset
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((139902, 50, 4), (99930, 50, 4), (139902, 1), (99930, 1))

In [15]:
stats = []

In [18]:
TCD = tempConvDecoder(**conf)
for _ in range(conf['eps']):
    print('epoch:', _)
    TCD.model.fit(
        X_train,
        y_train,
        epochs=1, 
        batch_size=conf['bs']
    )
    X_train, y_train = format_timeseries(
        dataset_paths, 
        conf['window'],
        conf['offset'],
        (0,4,1), 
        (0,1),
        sample_frac=.7
    )

R2s,rs = TCD.determine_fit(X_test, y_test)
stats.append([R2s, rs])

epoch: 0
Epoch 1/1
139902/139902 [==============================] - 8s - loss: 7.0494 - mean_squared_error: 79.5640     
epoch: 1
Epoch 1/1
139902/139902 [==============================] - 8s - loss: 5.7192 - mean_squared_error: 53.6295     


/home/laika117/Code/python/v1-decoder/tempConv.py:109: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  


In [19]:
print(stats)

[[[0.11697871163034224], [0.60457416487778293]], [[0.36218219726554701], [0.74612849063190245]]]


In [44]:
TCD.model.evaluate(TCD.X_test,TCD.y_test)

24608/24913 [============================>.] - ETA: 0s

[0.52790511442393295, 0.74992975554931163]

In [12]:
path = 'datasets/GRat31'
X_val, y_val = format_timeseries(
    path, 
    conf['window'],
    conf['offset'],
    (0,12,1), 
    (7,8),
    resample_data=conf['resample_data'],
    sample_size=conf['sample_size'],
    regressor=conf['regressor'],
    all_test=True
)

X_val.shape

['636429913515267697.hdf5', '636428029026710180.hdf5', '636430663717571697.hdf5', '636427282621202061.hdf5', '636431765535543697.hdf5', '636438658377315948.hdf5', '636439164041965948.hdf5', '636428953768193973.hdf5']


(493063, 60, 12)

In [13]:
dataset['train'][0].shape

(394453, 60, 12)